In [ ]:
class Solver:
    def __init__(self):
        pass

In [ ]:

from __future__ import print_function
from fenics import *
import mshr
import numpy as np

%matplotlib inline

totalTime = 5.0            # final time
num_steps = 500    # number of time steps
dt = totalTime / num_steps # time step size
eps = 0.01         # diffusion coefficient



class ChemicalReactionrate:
    
    reactionRate = 10.0           # random value of reaction rate
    
    def __init__(self, atmosphericTemp=300, cloudCover=0.5, solarElevation=0.79):
        self.atmosphericTemp = atmosphericTemp
        self.cloudCover = cloudCover
        self.solarElevation = solarElevation
        
        
    def reactionRate(self):
        """Defining the chemical reaction coefficient """ 
        k1 = 1/60*(0.5699-(9.056*(10**3)*(90-self.solarElevation))**2.546)*(1-0.75*(self.cloudCover/8)**3.4)
        k2 = 1.325*10**6*exp(-1430/self.atmosphericTemp)
        return k2/k1      

ChemicalReactionrateObject = ChemicalReactionrate()
reactionRateValue = ChemicalReactionrateObject.reactionRate()



class Domain:
    
    grid = mshr.Rectangle(Point(0, 0), Point(2.5, 2.5))
    mesh = mshr.generate_mesh(grid, 64)
    
    # Define function space for velocity
    velocityFunctionSpace = VectorFunctionSpace(mesh, 'P', 2)

    # Define function space for system of concentrations
    P1 = FiniteElement('P', triangle, 1)
    element = MixedElement(P1, P1, P1, P1)
    concentrationFunctionSpace = FunctionSpace(mesh, element)


#     def __init__(self, NO, O_3, NO_2, SO_2):
        
#         self.NO = NO
#         self.O_3 = O_3
#         self.NO_2 = NO_2
#         self.SO_2 = SO_2

        
    def concentrationTestFunction(self):   
        
        # Define test functions
        concentrationTestFunction = TestFunctions(Domain.concentrationFunctionSpace)
        return concentrationTestFunction
    
    def velocityFunction(self):
        # Define functions for velocity
        velocityFunction = Function(Domain.velocityFunctionSpace)
        return velocityFunction
        
    def concentrationFunction(self):
        ''' Define functions for Concentrations'''         
        concentrationFunction = Function(V)
        return concentrationFunction
    
    def concentrationSplit(self):
        # Split system functions to access components
        concentrationSplit = split(concentrationFunction)
        
        
        
    
class SourceDefinition:
        # Define source terms
        f_1 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.1,2)<0.05*0.05 ? 0.1 : 0',
                         degree=1)
        f_2 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.3,2)<0.05*0.05 ? 0.1 : 0',
                         degree=1)
        f_3 = Constant(0)

        f_4 = Expression('pow(x[0]-0.1,2)+pow(x[1]-0.5,2)<0.05*0.05 ? 0.1 : 0',
                         degree=1)

        
        
class Solver:

    # Define expressions used in variational forms
    k = Constant(dt)
    reactionRateValue = Constant(reactionRateValue)
    eps = Constant(eps)

    
    def concentrationSplitInitialTime(self):
        # Split system functions to access components
        NO_Initial, O_3_Initial, NO_2_Initial, SO_2_Initial = split(concentrationFunction)
        
    def concentrationSplitFinalTime(self):
        # Split system functions to access components
        NO_Final, O_3_Final, NO_2_Final, SO_2_Final = split(concentrationFunction)
        

    
    F = ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx \
        + eps*dot(grad(u_1), grad(v_1))*dx + reactionRateValue*u_1*u_2*v_1*dx \
        + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx \
        + eps*dot(grad(u_2), grad(v_2))*dx + reactionRateValue*u_1*u_2*v_2*dx \
        + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx \
        + eps*dot(grad(u_3), grad(v_3))*dx - reactionRateValue*u_1*u_2*v_3*dx + reactionRateValue*u_3*v_3*dx \
        +((u_4 - u_n4) / k)*v_4*dx + dot(w, grad(u_4))*v_4*dx \
        + eps*dot(grad(u_4), grad(v_4))*dx - f_1*v_1*dx - f_2*v_2*dx \
        - f_3*v_3*dx - f_4*v_4*dx


    
    



# Create time series for reading velocity data
timeseries_w = TimeSeries('Downloads/wind_data21')
#timeseries_w = TimeSeries('navier_stokes_cylinder/velocity_series')
  
    
# Create VTK files for visualization output
vtkfile_u_1 = File('reaction_system/u_1.pvd')
vtkfile_u_2 = File('reaction_system/u_2.pvd')
vtkfile_u_3 = File('reaction_system/u_3.pvd')
vtkfile_u_4 = File('reaction_system/u_4.pvd')


# Create progress bar
progress = Progress('Time-stepping')
#set_log_level(PROGRESS)

# Time-stepping
t = 0
for n in range(num_steps):

    # Update current time
    t += dt

    # Read velocity from file
    timeseries_w.retrieve(w.vector(), t)

    # Solve variational problem for time step
    solve(F == 0, u)

    # Save solution to file (VTK)
    _u_1, _u_2, _u_3, _u_4 = u.split()
    vtkfile_u_1 << (_u_1, t)
    vtkfile_u_2 << (_u_2, t)
    vtkfile_u_3 << (_u_3, t)
    vtkfile_u_4 << (_u_4, t)
    
    # Update previous solution
    u_n.assign(u)

    # Update progress bar
#progress.update(t / T)

# Hold plot
#interactive()

In [ ]:


from fenics import *

class Plume_sim:
    """ 
    Plume transport and reaction in a 2D Domain with homogenous 
    Neumann boundary conditions for u1, u2, u3, and u4; that is
    du_i/dn = 0 for 1 = 1, 2, 3, 4.
    
    ===========         ===================================
    Attributes                       Description
    ===========         ===================================
    cld                 cloud cover
    T                   Atmospheric temperature
    s_elev              solar elevation    
    nx                  number of cells along the x-axis
    ny                  number of cells along the y-axis
           
    """
    
    from math import exp    
    
    def __init__(self, cld, T, s_elev, nx = 100, ny = 100):
        
        """Intialize plume simulation in the domain """
        
        self.cld = cld
        self.T = T
        self.s_elev = s_elev
        self.nx = nx
        self.ny = ny
        
                
    def mesh_grid(self):
        """ Creates a 2D mesh given the number of cells along 
        the x and y co-ordinates, nx and ny """
        return RectangleMesh(Point(0, 0), Point(1, 1), nx, ny)
        
    def velocity_f(self):
        """ Define function space for the velocity """
        W = VectorFunctionSpace(self.mesh_grid, 'P', 2)
        return Function(W)
       
    def conc(self):
        """ Define function space for the system of concentration """
        P1 = FiniteElement('P', triangle, 1)
        element = MixedElement([P1, P1, P1, P1])
        return FunctionSpace(self.mesh_grid, element)
    
    def rx_const(self):
        """Defining the chemical reaction constant """ 
        k1 = 1/60*(0.5699-(9.056*10**3(90-s_elev))**2.546)*(1-0.75*(cld/8)**3.4)
        k2 = 1.325*10**6*exp(-1430/T)
        return k2/k1      
    
    def u(self):
        return Function(self.conc)
        
    def u_n(self):
        return Function(self.conc)
    
    def var_form(self):
        """ Define concentration Functions and test  """
        v_1, v_2, v_3, v_4 = TestFunctions(self.conc)
        
        #Split concentration function to access components
        u_1, u_2, u_3, u_4 = split(self.u)
        u_n1, u_n2, u_n3, u_n4 = split(self.u_n)
        
        #Defining source terms
        f_1 = ""
        f_2 = ""
        f_3 = Constant(0)
        f_4 = ""
                    
        #Defining the variational problem
        return ((u_1 - u_n1) / k)*v_1*dx + dot(w, grad(u_1))*v_1*dx               + eps*dot(grad(u_1), grad(v_1))*dx + self.rx_const*u_1*u_2*v_1*dx                + ((u_2 - u_n2) / k)*v_2*dx + dot(w, grad(u_2))*v_2*dx               + eps*dot(grad(u_2), grad(v_2))*dx + self.rx_const*u_1*u_2*v_2*dx                + ((u_3 - u_n3) / k)*v_3*dx + dot(w, grad(u_3))*v_3*dx               + eps*dot(grad(u_3), grad(v_3))*dx - self.rx_const*u_1*u_2*v_3*dx + self.rx_const*u_3*v_3*dx               +((u_4 - u_n4) / k)*v_4*dx + dot(w, grad(u_4))*v_4*dx               + eps*dot(grad(u_4), grad(v_4))*dx - f_1*v_1*dx - f_2*v_2*dx               - f_3*v_3*dx - f_4*v_4*dx
                
                
def run_solver():
    
    # Creating an instance of Plume_sim class
    Sim = Plume_sim(10, 283, 45)
    
    # Create time series for reading velocity data
    timeseries_w = TimeSeries('')
       
    # Create VTK files for visualization output
    vtkfile_u_1 = File('')
    vtkfile_u_2 = FIle('')
    vtkfile_u_3 = FIle('')
    vtkfile_u_4 = FIle('')
    
    t = 0
    for n in range(n_steps):
        
        # Update current time
        t += dt      
        
        # Read velocity from file
        timeseries_w.retrieve(w.vector(), t)
        
        # Solve variational problem for the step
        solve(Sim.var_form() ==0, Sim.u())      
        
        # Save solution to file (VTK)
        _u_1, _u_2, _u_3, _u_4 = Sim.u().split()
        vtkfile_u_1 << (_u_1, t)
        vtkfile_u_2 << (_u_2, t)
        vtkfile_u_3 << (_u_3, t)
        vtkfile_u_4 << (_u_4, t)
        
        #update previous solution
        Sim.u_n().assign(Sim.u())    
        
        
if __name__ == '__main__':
    run_solver()  
    


In [ ]:
import numpy as np

class ChemicalReactionrate:
    
    reactionRate = 10.0           # reaction rate
    
    def __init__(self, solarElevation = 0.79, cloudCover = 0.5, atmosphericTemp = 290):
        self.atmosphericTemp = atmosphericTemp
        self.cloudCover = cloudCover
        self.solarElevation = solarElevation
        
        
#     def reactionRate(self):
#         """Defining the chemical reaction constant """ 
#         k1 = 1/60*(0.5699-(9.056*(10**3)*(90-self.solarElevation))**2.546)*(1-0.75*(self.cloudCover/8)**3.4)
#         k2 = 1.325*10**6*np.exp(-1430/self.atmosphericTemp)
#         reactionRate1 = k2/k1
#         return reactionRate1

    def rt(self):
        return self.atmosphericTemp*20
        

In [ ]:
ChemicalReactionrate

In [ ]:
ChemicalReactionrate.reactionRate

In [ ]:
# h = ChemicalReactionrate.reactionRate(1, 10, 300)
# print(h)
#p = ChemicalReactionrate()
h = ChemicalReactionrate.reactionRate
print(h)
r = p.rt()
print(r)

In [3]:
from __future__ import print_function
from fenics import *
import mshr
import numpy as np


class Domain:
    
    grid = mshr.Rectangle(Point(0, 0), Point(2.5, 2.5))
    mesh = mshr.generate_mesh(grid, 64)
    
    # Define function space for velocity
    velocityFunctionSpace = VectorFunctionSpace(mesh, 'P', 2)

    # Define function space for system of concentrations
    P1 = FiniteElement('P', triangle, 1)
    element = MixedElement(P1, P1, P1, P1)
    concentrationFunctionSpace = FunctionSpace(mesh, element)


#     def __init__(self, NO, O_3, NO_2, SO_2):
        
#         self.NO = NO
#         self.O_3 = O_3
#         self.NO_2 = NO_2
#         self.SO_2 = SO_2

        
    def concentrationTestFunction(self):   
        
        # Define test functions
        concentrationTestFunction = TestFunctions(Domain.concentrationFunctionSpace)
        return concentrationTestFunction
    
    def velocityFunction(self):
        # Define functions for velocity
        velocityFunction = Function(Domain.velocityFunctionSpace)
        return velocityFunction
        
    def concentrationFunction(self):
        ''' Define functions for Concentrations'''         
        concentrationFunction = Function(V)
        return concentrationFunction
    
#     def concentrationSplitInitialTime(self):
#         # Split system functions to access components
#         NO_Initial, O_3_Initial, NO_2_Initial, SO_2_Initial = split(concentrationFunction)
        
#     def concentrationSplitFinalTime(self):
#         # Split system functions to access components
#         NO_Final, O_3_Final, NO_2_Final, SO_2_Final = split(concentrationFunction)
        
    def concentrationSplit(self):
        # Split system functions to access components
        concentrationSplit = split(concentrationFunction)
        